In [1]:
!pip install transformers
!pip install pytorch
!pip install pytorch-transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 34.1 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-wlpjaa3d/pytorch_8f218e9ddc484782ae215f8e1e6b3dcc/setup.py'"'"'; __file__='"'"'/tmp/pip-install-wlpjaa3d/pytorch_8f21

In [2]:
import numpy as np
import pandas as pd
import re
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
import transformers
import torch
import tensorflow as tf
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

import transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import json
import gc

In [3]:
!gdown --id 16PkXbMaFEZQjV_bCGgq9n_R5ZQq1V1bp
train = pd.read_csv('df_train.csv')
#test = pd.read_csv('../input/scan-classification-challange/df_test.csv', index_col=0)
train.drop_duplicates(subset={'text'}, inplace=True) 
train.sample(3)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16PkXbMaFEZQjV_bCGgq9n_R5ZQq1V1bp
To: /content/df_train.csv
100% 16.7M/16.7M [00:00<00:00, 41.8MB/s]


,text,class
6212,Об этом во вторник сообщает пресс-служба СУ СК...,O
28590,Антикоррупционной службой изобличены заместите...,O
13115,"По данным ФСБ, с 2017 по 2018 год мужчина по з...",O


In [6]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [7]:
# Применяем get_dumies для таргета
train_class = pd.get_dummies(train, columns=['class'])
train_class.sample(3)

,text,class_O,class_Авиакатастрофа,class_Акционерный конфликт,"class_Беспорядки, конфликты и войны",class_Взрыв газа,class_Взятки,class_Возбуждение уголовного дела,class_Выемки,class_Жалобы потребителей,...,class_Убийство,class_Убытки,class_Уголовное задержание,class_Уголовное обвинение,class_Утечка нефти,class_Финансовые затруднения,"class_Чрезвычайные происшествия, чрезвычайные ситуации",class_Экономические преступления,class_Экстремизм,class_Ядерный взрыв
49913,"У него очень серьезные проблемы с климатом, но...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45613,"Директор НАБУ Артем Сытник заявлял, что у дете...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51810,Активисту вменяют клевету в отношении мэра гор...,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [8]:
train['labels'] = train_class.iloc[:, 1:].values.tolist()
train.sample(3)

,text,class,labels
27945,"КОРР.: Презентация книги ""Кизляр – Первомайско...",O,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
58182,"Ольга Мамыкина отмечает, что ""если у человека ...",O,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
32555,Всего за 11 месяцев в Новосибирской области вы...,Противодействие коррупции,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer

MAX_LEN = 390
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 2e-05

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
roberta = AutoModel.from_pretrained('xlm-roberta-base')

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
from torch import nn
#from transformers import BertModel
from torch.utils.data import Dataset, DataLoader

class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            #pad_to_max_length=True, # Устарел
            padding= 'max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [11]:
# Создаем dataset и dataloader для нейронной сети

train_size = 0.8
train_data=train.sample(frac=train_size,random_state=200)
test_data=train.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (59402, 3)
TRAIN Dataset: (47522, 3)
TEST Dataset: (11880, 3)


In [12]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
# замораживаем слои

#for param in distilbert.parameters():
for param in rubert.parameters():
    param.requires_grad = False

In [14]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [15]:
# Создаем Class для нашей модели
# тут добавляем слой Dropuot чтоб не переобучиться, а также 2 Linear

from pathlib import Path
class DistilBERTClass(torch.nn.Module):
    def __init__(self, num_labels=50):
        super(DistilBERTClass, self).__init__()
        self.num_labels = num_labels
        self.roberta = roberta
        self.dropout = nn.Dropout(0.1)
        #self.distilbert = distilbert
        #self.l1 = AutoModel.from_pretrained("cointegrated/rubert-tiny")
                        
        self.relu = nn.ReLU()
        self.pre_classifier = nn.Linear(768, 768)
        self.classifier = nn.Linear(768, 50)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = bert_output.last_hidden_state[:, 0, :] # hidden_state = output_1[0]
        #pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(last_hidden_state)
        pooler = self.relu(pooler)
        #pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (roberta): XLMRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [16]:
from transformers import get_linear_schedule_with_warmup

def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE, eps = 1e-8)

#epochs = 4
total_steps = len(training_loader) * EPOCHS
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [17]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()

In [18]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Epoch: 0, Loss:  0.6965292096138


0it [00:00, ?it/s]

Epoch: 1, Loss:  0.013347476720809937


0it [00:00, ?it/s]

Epoch: 2, Loss:  0.003195849945768714


0it [00:00, ?it/s]

Epoch: 3, Loss:  0.004318286664783955


0it [00:00, ?it/s]

Epoch: 4, Loss:  0.007202259264886379


In [19]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [20]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

0it [00:00, ?it/s]

In [22]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))
f1_m = f1_score(targets, final_outputs, average='weighted', zero_division = 1)
print(f"Hamming Loss = {val_hamming_loss}")
print(f'F_1 = {f1_m}')

Hamming Loss = 0.0030454545454545456
F_1 = 0.9157790419239747
